# Open Glassdoor Jobs

In [27]:
from scrapy import *

In [3]:
links = crawl_gd()

pulling links from page 1
Got 30 links from page 1
pulling links from page 2
Got 26 links from page 2
pulling links from page 3
Got 19 links from page 3
pulling links from page 4
Got 22 links from page 4
pulling links from page 5
Got 0 links from page 5


In [4]:
links[0]

'http://www.glassdoor.com/partner/jobListing.htm?pos=101&ao=86735&s=58&guid=00000150c0873ce3b31e3f136e4c61ff&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&cb=1446339100169&jobListingId=1625651228'

In [109]:
soup = get_soup(links[0])

In [110]:
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)


In [111]:
first_line = text.split('\n')[0]

In [112]:
chunks = first_line.split(' Job in ')

In [113]:
job_title = chunks[0]
location = " ".join(chunks[1].split(' ')[0:2])

In [114]:
job_title

'Clarity Solution Group Data Scientist'

In [115]:
location

'Chicago, IL'